In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib as mpl
import matplotlib.cm as cm
import cPickle as pickle
import helpers
import plots
import pandas as pd
import glob
reload(plots)
reload(helpers)

<module 'helpers' from 'helpers.pyc'>

# Figure preliminaries notebook

The aim of this notebook is just to collect together the relevant data into a single dataframe to speed up plotting in the other notebooks. 

The notebook loads the traces and creates a dataframe for each, before concatenating these dataframes, storing plotting information (colour, marker style) and pickling the result. This can then be loaded in the other notebooks.

In [5]:
reload(helpers)

## Make list of traces and filenames
tracelist = glob.glob('traces/*trace.p')
filenamelist = []
for trace in tracelist:
    filename = trace.split('/')[0]+'/'+trace.split('/')[1].replace('trace', 'filenames')
    filenamelist.append(filename)
    
## Build dataframe of posterior means
thetas = ['theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'theta7', 'theta8']
dfs = []
for i, tracename in enumerate(tracelist):
    print 'Loading traces for %s' %(tracename)
    trace = pickle.load(open(tracename, 'rb'))
    subjs = pickle.load(open(filenamelist[i], 'rb'))
    
    df = helpers.make_single_pm_df(trace, subjs, thetas)
    dfs.append(df)

Loading traces for traces/LiCl_32.0_R_L_trace.p
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_3_8_2014-10-17.B0103.CSV
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_9_8_2014-10-27.B0109.CSV
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_6_8_2014-10-17.B0106.CSV
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_5_8_2014-11-03.B0105.CSV
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_7_8_2014-11-03.B0107.CSV
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_12_8_2014-10-27.B0112.CSV
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_8_8_2014-11-03.B0108.CSV
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_4_8_2014-11-03.B0104.CSV
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_11_8_2014-10-17.B0111.CSV
new_all_data/LiCl_32.0_R_L/LiCl_32.0_R_L_10_8_2014-10-27.B0110.CSV
Loading traces for traces/PYY_1.5_A_D_trace.p
new_all_data/PYY_1.5_A_D/PYY_1.5_A_D_9_8_2015-02-17.B0109.CSV
new_all_data/PYY_1.5_A_D/PYY_1.5_A_D_7_8_2015-02-17.B0107.CSV
new_all_data/PYY_1.5_A_D/PYY_1.5_A_D_11_8_2015-02-17.B0111.CSV
new_all_data/PYY_1.5_A_D/PYY_1.5_A_D_13_8_2015-02-09.B01

new_all_data/GLP-1_30.0_A_D/GLP-1_30.0_A_D_4_8_2015-02-23.B0104.CSV
new_all_data/GLP-1_30.0_A_D/GLP-1_30.0_A_D_14_8_2015-03-02.B0114.CSV
new_all_data/GLP-1_30.0_A_D/GLP-1_30.0_A_D_7_8_2015-03-09.B0107.CSV
new_all_data/GLP-1_30.0_A_D/GLP-1_30.0_A_D_5_8_2015-03-09.B0105.CSV
new_all_data/GLP-1_30.0_A_D/GLP-1_30.0_A_D_6_8_2015-03-02.B0106.CSV
new_all_data/GLP-1_30.0_A_D/GLP-1_30.0_A_D_9_8_2015-03-09.B0109.CSV
new_all_data/GLP-1_30.0_A_D/GLP-1_30.0_A_D_11_8_2015-03-09.B0111.CSV
new_all_data/GLP-1_30.0_A_D/GLP-1_30.0_A_D_13_8_2015-03-09.B0113.CSV
Loading traces for traces/saline_0.0_A_D_trace.p
new_all_data/saline_0.0_A_D/saline_0.0_A_D_8_8_2015-03-09.B0108.CSV
new_all_data/saline_0.0_A_D/saline_0.0_A_D_11_8_2015-03-02.B0111.CSV
new_all_data/saline_0.0_A_D/saline_0.0_A_D_4_10_2015-11-23.B0104.CSV
new_all_data/saline_0.0_A_D/saline_0.0_A_D_10_8_2015-02-17.B0110.CSV
new_all_data/saline_0.0_A_D/saline_0.0_A_D_3_8_2015-02-04.B0103.CSV
new_all_data/saline_0.0_A_D/saline_0.0_A_D_6_8_2015-02-09.B01

IOError: [Errno 2] No such file or directory: 'new_all_data/PYY_7.5_A_D/PYY_7.5_A_D_5_8_2015-02-09.B0105.CSV'

In [7]:
## Concatenate dataframes
full_df = pd.concat(dfs, ignore_index=True)

## Convert to numeric dtype
to_convert = thetas + ['dose', 'id', 'duration', 'rate']
full_df[to_convert] = full_df[to_convert].apply(pd.to_numeric)

## Store marker colour for drugs
def get_drug_colour(row):
    data = [row['drug'], row['dose'], row['adlib'], row['period']]
    return helpers.get_colour(data)
    
full_df['drug_c'] = full_df.apply(get_drug_colour, axis=1)

## Store marker colour for normalised intake
ratemin = full_df['rate'].min()
ratemax = full_df['rate'].max()

rate_norm = mpl.colors.Normalize(vmin = ratemin,
                                 vmax = ratemax)

cmap = cm.ScalarMappable(norm=rate_norm, cmap=cm.plasma)

def get_rate_colour(row):
    return cmap.to_rgba(row['rate'])

full_df['rate_c'] = full_df.apply(get_rate_colour, axis=1)

## Store marker style
def get_ms(row):
    if row['period'] == 'D':
        return '^'
    
    else:
        return 'o'
    
full_df['ms'] = full_df.apply(get_ms, axis=1)

theta1                                               -2.01676
theta2                                               -2.31717
theta3                                               -2.86513
theta4                                               0.921754
theta5                                                1.35661
theta6                                               -1.92572
theta7                                                  3.299
theta8                                                 2.5411
drug                                                     LiCl
dose                                                       32
adlib                                                       R
period                                                      L
id                                                          3
duration                                                    8
filename                                 2014-10-17.B0103.CSV
filepath                           new_all_data/LiCl_32.0_R_L
rate    

Name: 115, dtype: object
theta1                                               -2.04855
theta2                                               -2.22699
theta3                                               -2.74649
theta4                                                1.24461
theta5                                               0.459588
theta6                                               -1.50953
theta7                                                2.96123
theta8                                                 2.1657
drug                                                    GLP-1
dose                                                       30
adlib                                                       A
period                                                      D
id                                                         14
duration                                                    8
filename                                 2015-03-02.B0114.CSV
filepath                          new_all_dat

In [8]:
## Pickle the result
pickle.dump(full_df, open('pm_df.p', 'wb'))

In [ ]:
print full_df